In [1]:
!pip install gradio --quiet

import gradio as gr
import time

print(gr.__version__)


     |████████████████████████████████| 2.2 MB 5.1 MB/s 
     |████████████████████████████████| 206 kB 36.1 MB/s 
     |████████████████████████████████| 1.9 MB 50.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 961 kB 40.8 MB/s 
     |████████████████████████████████| 3.2 MB 38.5 MB/s 
2.2.8


In [12]:
!pip install transformers --quiet
!pip install allennlp==1.0.0 allennlp-models==1.0.0 --quiet
#!pip install --user gradio

from urllib import request
from bs4 import BeautifulSoup as bs
import re
import nltk
import heapq
from transformers import pipeline
from urllib.parse import urlparse
from urllib.request import urlopen, Request


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from allennlp.predictors.predictor import Predictor

summarizer = pipeline("summarization") # model="t5-small"
# predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-model-2020.03.19.tar.gz")


100%|██████████| 45959361/45959361 [00:01<00:00, 40537313.27B/s]


In [28]:
def url_validator(x):

    try:

        result = urlparse(x)
        return all([result.scheme, result.netloc])

    except:

        return False


#First compile everything into a function
def main(url_or_plain_text, question, bullet_points):

    if url_validator(url_or_plain_text):

        url = url_or_plain_text
        TotalContent = ""

        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        htmlDoc = urlopen(req).read()

        soupObj = bs(htmlDoc)
        paragraphContents = soupObj.findAll('p')

        for content in paragraphContents:

            TotalContent += content.text

    else:

        TotalContent = url_or_plain_text


    total_cleaned_content = re.sub(r'\[[0-9]*\]', ' ', TotalContent)
    total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

    sentence_tokens = nltk.sent_tokenize(total_cleaned_content)

    total_cleaned_content = re.sub(r'[^a-zA-Z]', ' ', total_cleaned_content)
    total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

    #print(sentence_tokens)

    words_tokens = nltk.word_tokenize(total_cleaned_content)

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}

    for word in words_tokens:

        if word not in stopwords:

            if word not in word_frequencies.keys():

                word_frequencies[word] = 1
            else:

                word_frequencies[word] += 1


    #print(words_tokens)

    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

    #print(word_frequencies)

    sentence_scores = {}

    for sentence in sentence_tokens:

        for word in nltk.word_tokenize(sentence.lower()):

            if word in word_frequencies.keys():

                if (len(sentence.split(' '))) < 30:

                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_frequencies[word]
                    else:
                        sentence_scores[sentence] += word_frequencies[word]


    points = heapq.nlargest(20, sentence_scores, key=sentence_scores.get)

    total_text = ""

    for i in points:

        total_text += i

    bullet_point_string = """"""

    for i in range(bullet_points):
        
        bullet_point_string += "\n - " + points[i]

    

    summary = summarizer(total_text, max_length=128, min_length=64, do_sample=False)

    result=predictor.predict(
    passage=TotalContent,
    question=question
    )

    return summary[0]["summary_text"], bullet_point_string, result['best_span_str']


In [ ]:
gr.Interface(
  fn=main, 
  inputs=[gr.inputs.Textbox(lines=5, placeholder="A text consisting of 20 or more sentences is recommended"), 
          gr.inputs.Textbox(lines=2), 
          gr.inputs.Slider(0, 10, step=1)],
  outputs=["text", "text", "text"], 
  theme="huggingface").launch(debug=False, share=True)